<a href="https://colab.research.google.com/github/m-mehdi/Python101/blob/master/ApacheSpark_02_CIMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apache Spark Dataframe Exercise

---



In this exercise, we are going to get some insights on stock market data. We use `walmart_stock.csv` file as our dataset to analyse the data.

### 1- Create an Apache Spark *Session*

In [2]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 75kB/s 
     |████████████████████████████████| 204kB 35.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=b2b55bbeedb02cad88eaa45b286dd1542431c61e763b9cd50d3d5f2f76780bce
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [5]:
spark

## 2- Load the `walmart_stock.csv` file into a dataframe and infer the data schema

In [6]:
df = spark.read.csv('walmart_stock.csv', header=True, inferSchema=True)

## 3- Display the column names and print the dataframe schema

In [7]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [8]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



## 4- Print out the first five rows of the data

In [9]:
df.head(5)

[Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

## 5- Use `describe()` method to get statistical information on the data 

In [12]:
df.describe(['Open','Close']).show()

+-------+------------------+-----------------+
|summary|              Open|            Close|
+-------+------------------+-----------------+
|  count|              1258|             1258|
|   mean| 72.35785375357709|72.38844998012726|
| stddev|  6.76809024470826|6.756859163732991|
|    min|56.389998999999996|        56.419998|
|    max|         90.800003|        90.470001|
+-------+------------------+-----------------+



## 6- Use `format_number` function to format the numbers for just showing up to two decimal places. 
[format_number() documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=format_number#pyspark.sql.functions.format_number)

In [15]:
des_result = df.describe()
from pyspark.sql.functions import format_number

In [16]:
des_result.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [24]:
des_result.select(format_number('Open'.cast('float'),2)).show()

AttributeError: ignored

In [25]:
des_result['Open']

Column<b'Open'>

In [26]:
des_result.select(des_result['Summary'],
                  format_number(des_result['Open'].cast('float'),2).alias('Open'),
                  format_number(des_result['High'].cast('float'),2).alias('High'),
                  format_number(des_result['Low'].cast('float'),2).alias('Low'),
                  format_number(des_result['Close'].cast('float'),2).alias('Close'),
                  des_result['Volume'].cast('int').alias('Volume'),
                  format_number(des_result['Adj Close'].cast('float'),2).alias('Adj Close'),).show()

+-------+--------+--------+--------+--------+--------+---------+
|Summary|    Open|    High|     Low|   Close|  Volume|Adj Close|
+-------+--------+--------+--------+--------+--------+---------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258| 1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|    67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|     6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|    50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|    84.91|
+-------+--------+--------+--------+--------+--------+---------+



## 7- Create a new coulmn called HV Ratio on a new dataframe that returns the ratio of the High Price versus volume of stock traded for a day.

In [27]:
HV_df =df.withColumn("HV Ratio", df['High']/df['Volume'])

In [28]:
HV_df.select('HV Ratio').show(5)

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
+--------------------+
only showing top 5 rows



## 8- What day had the Peak High in Price?

In [30]:
df.orderBy(df['High'].desc()).head()[0]

'2015-01-13'

## 9-What is the mean of the Close column?

In [31]:
from pyspark.sql.functions import mean
df.select(mean('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [32]:
df.agg({'Close':'mean'}).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



## 10- How many days was the Close lower than USD 70?

In [33]:
df.filter('Close < 70').count()

397

## 11-What percentage of the time was the High greater than 80 USD ?
#### In other words, (Number of High Days>80)/(Total Days in the dataframe)

In [36]:
(df.filter(df['High']>80).count()/df.count())*100

9.141494435612083

## 12-What is the correlation between High and Volume?

In [37]:
from pyspark.sql.functions import corr
df.select(corr('High','Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



## 13- What is the max High per year (use GroupBy)?

In [39]:
from pyspark.sql.functions import max, year
year_df = df.withColumn('Year',year(df['Date']))
max_df=year_df.groupBy('Year').max()
max_df.select('Year','max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



## 14- What is the average Close for each Calendar Month (close price for Jan,Feb, Mar, etc)?

In [40]:
from pyspark.sql.functions import month
month_df = df.withColumn('Month', month('Date'))
month_avg = month_df.select('Month','Close').groupBy('Month').mean()

In [42]:
month_avg.select('Month','avg(Close)').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

